# Filtering

Function to develop/test the filtering based on a checkerboard of white and grey squares.

## Initialization and imports

Imports libraries and initializes the communication with Thymio through USB port.

### Look-up table for ports:
Replace `/dev/cu.usbmodem142101` (or current port value) below by the correct computer port:

Lucas: `\\.\COM3` for USB-cable and `\\.\COM4` for USB-dongle

Emma: ` `

Elise: ` `

Océane: ` `

In [ ]:
import os
import sys
import time
import serial
from math import *

# Variable to know if Thymio has already been connected before or not (== 1 if it is the case)
try:
    CONNECC
except NameError:
    CONNECC = 0
    print('Thymio will be connected.')
else:
    print('Thymio has already been connected to the computer')

# Adding the src folder in the current directory as it contains the script
# with the Thymio class
#sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

from Thymio import Thymio

# Print the path to Python3 executable
# print(sys.executable)

if CONNECC == 0:
    CONNECC = 1
    th = Thymio.serial(port="\\.\COM3", refreshing_rate=0.1)
    time.sleep(1)

## (Odometry)

Important stuff are the coefficients for the odometry and the global structure of the code to make the Thymio moves towards the defined goal:

In [ ]:
counter = 0
VelLeft = 100
VelRight = -100
adjust = 1.9 #4.15
SpeedGain = adjust*0.0325 * 10**(-3)
timer = 1 * 10**(-3) # 1ms
b = 0.095 # distance between Thymio's wheels in m

X = 0
Y = 0
Theta = 0

t = 0

while counter < 1000:
    #a = time.time()
    counter += 1
    th.set_var("motor.right.target", VelRight)
    th.set_var("motor.left.target", VelLeft)

    DTheta = (VelLeft*SpeedGain - VelRight*SpeedGain)/(2*b)
    DS = (VelLeft + VelRight)/2
    
    Theta += DTheta
    X += DS*SpeedGain*cos(radians(Theta))
    Y += DS*SpeedGain*sin(radians(Theta))

    time.sleep(timer)
    #b = time.time()
    #t += b-a

#print(t)
th.set_var("motor.right.target", 0)
th.set_var("motor.left.target", 0)

print(X)
print(Y)
print(Theta)

## To do:

- Définir fonctions et classes utiles à l'ensemble du déplacement du Thymio
    - markers2reality traduit les coordonnées des cases du damier en cm DONE
    - Thymio contient la position du Thymio ainsi que permet de régler la vitesse des roues et récupérer les informations relatives aux différents capteurs DONE
    - Kalmanfilter est la classe qui contiendra les différentes matrices nécessaires au filtrage
        - Concernant le filtrage en lui-même: état X = (x, y, theta); A = matrice identité; B = odometry (change avec l'état du Thymio, car varie avec l'angle du Thymio, voir slides 19-20 cours sur la localisation) dont un exemple se trouve ci-dessus, les facteurs adjust et 0.0325 sont déduit par expériences, ils changeront probablement dû au chgt de la durée d'une boucle par rapport à l'exemple ci-dessus, il faudra donc préalablement dire que l'odométrie est parfaite, faire des tests pour voir si l'odométrie est satisfaisante ou non* (voir notes en bas); C = matrice identité si mesures == prédictions, sinon il faut changer les valeurs (c'est la dernière grande inconnue, je pense que ça se fait par intuitions premièrement, et ensuite à tâtons lors des tests, ne vous embêtez pas trop avec ça pour le moment, dîtes que c'est égal à 2 fois l'identité par exemple)
    - Si d'autres fonctions/classes vous semblent intéressantes à créer vous gêner pas évidemment
- Créer la boucle principale, celle-ci doit effectuer les tâches suivantes dans cet ordre:
    - Initialiser la classe Thymio avec la start position voulue
    - Déterminer la vitesse qu'il faut appliquer aux roues afin d'atteindre le prochain objectif (qu'il faut définir évidemment, ça peut simplement se faire par coordonnées des cases évidemment)
    - Bouger le thymio 
    - Appliquer le filtrage pour déterminer la nouvelle position du Thymio
    - Si le goal est considéré comme atteint (|X - goal| < epsilon, tel que epsilon << 1), alors définir goal suivant, sauf si c'était la fin, dans ce cas, Pablo commence à faire du break dance.
    
*NOTES*: lorsque vous aurez tout coder, il faudra tester le tout en disant que l'odométrie est parfaite, mais en faisant quand même tous les calculs de filtrage!!! Pour déterminer les coefficients d'ajustement de l'odométrie qui dépendent forcément de la durée d'une boucle complète!
Evidemment, ne suivez pas bêtement cette liste de choses à faire si vous pensez qu'il est possible de faire mieux, je ne suis pas Dieu, j'ai pratiquement rien foutu hier soir.
    

## Definitions

### Functions

#### markers2reality:

This function gives the position in real coordinates (cm) of the given markers coordinates (matrix coordinates). The checkerboard pattern has 6cm sided squares:

In [1]:
def markers2reality(mx, my):
    x = (2*mx + 1)*3
    y = (2*my + 1)*3
    return (x,y)

### Classes

#### Thymio

This class designates the robot, including its state (x, y and theta) and the different functions that control its displacement:

In [33]:
class Thymio:
    def __init__(self, x, y): # x and y are in cm, theta in degrees
        self.x = x
        self.y = y
        self.theta = 90
        self.ground = [0,0]
        self.front = [0,0,0,0,0]
    
    def set_pos(self, x, y, theta):
        self.x = x
        self.y = y
        self.theta = theta
    
    def get_ground(self):
        self.ground = th["prox.ground.delta"] # à déterminer s'il vaut mieux choisir ground.ambiant, ground.delta
        return self.ground                    # ou bien encore ground.... (le dernier des trois possibles, voir les variables Aseba)
        
    def get_front(self):
        self.front = th["prox.horizontal"][0:4]
        return self.front
        
    def pos(self):
        return [self.x, self.y, self.theta]
    
    def move(self, u): # soit simplement set la vitesse des roues, possible de faire l'odometry ici aussi je pense
        
        # Move thymio depending on the control inputs u
        th.set_var("motor.left.target", u[0])
        the.set_var("motor.right.target", u[1])
        
    def nopuedo(self): # je sais pas encore ce que c'est, mais ce sera quelque chose j'imagine, jpp, arg, prout, pouet
        
    
        
    def __repr__(self): # called when print(Thymio) is used
        return "(x, y, theta) = (%.2f, %.2f, %.1f)" %(self.x, self.y, self.theta)

In [30]:
a = 5.3
b = 2.65784
c = "x = %.2f, y = %.2f" %(a, b)
print(c)

x = 5.30, y = 2.66


#### Kalmanfilter

Extended Kalman filter called at each iteration after Thymio's movement. See lecture on Uncertainties slides 33-end. Previous lecture on Localisation might be useful as well, odometry slides 19-20 and intro. to uncertainties slides 23-37.

In [ ]:
class Kalmanfilter:
    def __init__(self, A, B, w, C1, C2, v, pred_mu, pred_Sigma, mu, Sigma):
        self.A = A # jacobian, constant
        self.B = B # odometry matrix, change with theta of Thymio
        self.w = w # displacement noise, constant
        self.C1 = C1 # measurement matrix measurements == predictions
        self.C2 = C2 # measurement matrix when measurements =/= predictions
        self.v = v # measurement noise, constant
        self.mu = mu # current mean, aka current state
        self.Sigma = 0 # current covariance matrix
        
    def predict(self): # first step
        
    def innovation(self): # second step
        
    def aposteriori(self): # final step